# Compute Frechet Inception Distance for a model

To do this, one needs an "Inception Net". A discriminator that discriminates between real and fake samples.

In [1]:
import os
import sys
sys.path.append(os.pardir)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch 

In [3]:
import h5py

In [20]:
import gc

In [4]:
from glob import glob

In [5]:
from utils.utils import load_netE, load_netG

In [6]:
from models.nets_16col_residual import _netG, _netD, _netE, weights_init

In [7]:
from data.dataset import bird_dataset

# load a generator

In [14]:
netG = load_netG('/home/gagan/data/zudeep/models/nz_12_residual/2021-05-05_10-41-37.547480/netG_epoch_5.pth', nz=12, ngf=128, cuda=True, resnet=True)

In [15]:
print(netG)

_netG(
  (lnormlist): ModuleList(
    (0): LayerNorm((1024, 3, 3), eps=1e-05, elementwise_affine=True)
    (1): LayerNorm((512, 8, 4), eps=1e-05, elementwise_affine=True)
    (2): LayerNorm((512, 8, 4), eps=1e-05, elementwise_affine=True)
    (3): LayerNorm((512, 8, 4), eps=1e-05, elementwise_affine=True)
    (4): LayerNorm((256, 31, 5), eps=1e-05, elementwise_affine=True)
    (5): LayerNorm((256, 31, 5), eps=1e-05, elementwise_affine=True)
    (6): LayerNorm((256, 31, 5), eps=1e-05, elementwise_affine=True)
    (7): LayerNorm((128, 63, 8), eps=1e-05, elementwise_affine=True)
  )
  (relu): ReLU(inplace=True)
  (convs): ModuleList(
    (0): ConvTranspose2d(12, 1024, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(3, 1), padding=(1, 1), bias=False)
    (2): ConvTranspose2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ConvTranspose2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4):

# make a generators

In [18]:
netD = _netD(ndf=128, nc=1)
netD = netD.cuda()

In [19]:
print(netD)

_netD(
  (convs): ModuleList(
    (0): Conv2d(1, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): Conv2d(128, 256, kernel_size=(4, 3), stride=(2, 1), padding=(1, 0), bias=False)
    (2): Conv2d(256, 512, kernel_size=(4, 3), stride=(2, 1), padding=(2, 0), bias=False)
    (3): Conv2d(512, 1024, kernel_size=(8, 4), stride=(2, 1), padding=(1, 1), bias=False)
    (4): Conv2d(1024, 1, kernel_size=(6, 3), stride=(1, 1), bias=False)
  )
  (lns): ModuleList(
    (0): LayerNorm((128, 64, 8), eps=1e-05, elementwise_affine=True)
    (1): LayerNorm((256, 32, 6), eps=1e-05, elementwise_affine=True)
    (2): LayerNorm((512, 17, 4), eps=1e-05, elementwise_affine=True)
    (3): LayerNorm((1024, 6, 3), eps=1e-05, elementwise_affine=True)
  )
  (activations): ModuleList(
    (0): LeakyReLU(negative_slope=0.2)
    (1): LeakyReLU(negative_slope=0.2)
    (2): LeakyReLU(negative_slope=0.2)
    (3): LeakyReLU(negative_slope=0.2)
  )
)


In [21]:
del netD
gc.collect()

2073

In [ ]:
class _netD(nn.Module):
    def __init__(self, ndf,nc):
        super(_netD, self).__init__()
        self.convs = nn.ModuleList([
            nn.Conv2d(nc, ndf, kernel_size=4, stride=(2,2), padding=(1,1), bias=False),
            # size H = (129 +2 -4)/2 + 1 = 64, W = (16 +2 -4)/2 + 1 = 8
            nn.Conv2d(ndf, ndf * 2, kernel_size=(4, 3), stride=(2, 1), padding=(1, 0), bias=False),
            # size H = (64 +2 -4)/2 + 1 = 32, W = (8 -3) + 1 = 6
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size=(4,3), stride=(2,1), padding=(2,0), bias=False),
            # size H = (32 +4 -4)/2 + 1 = 17, W = (6 -3) + 1 = 4
            nn.Conv2d(ndf * 4, ndf * 8, kernel_size=(8,4), stride=(2,1), padding=1, bias=False),
            # H = (17 +2 -8)/2 + 1 = 6, W = (4 +2 -4) + 1 = 3
            nn.Conv2d(ndf * 8, 1, kernel_size=(6,3), stride=1, padding=0, bias=False),
            # H = 6-6 + 1 =1, W = (4 - 3) +1 =  
        ])
        self.lns = nn.ModuleList([nn.LayerNorm([ndf, 64, 8]),
                                  nn.LayerNorm([ndf * 2, 32, 6]),
                                  nn.LayerNorm([ndf * 4, 17, 4]),
                                  nn.LayerNorm([ndf * 8, 6, 3])])
        self.activations = nn.ModuleList([nn.LeakyReLU(0.2),nn.LeakyReLU(0.2),
                                    nn.LeakyReLU(0.2),nn.LeakyReLU(0.2)])

    def forward(self, x):
        for i in range(4):
            x = self.convs[i](x)
            x = self.lns[i](x)
            x = self.activations[i](x)
        y = x.clone()
        x = self.convs[-1](x)
        x = nn.Sigmoid()(x)
        output = x.view(-1,1)
        return output, y

# choose a set of birds as test dataset

In [ ]:
all_birds = os.listdir('/home/gagan/data/zudeep/bird_hdfs/')
all_birds

In [9]:
test_birds = glob('/home/gagan/data/zudeep/bird_hdfs/Box*')
len(test_birds)

226

In [10]:
test_birds[0]

'/home/gagan/data/zudeep/bird_hdfs/Box5_HP24'

In [11]:
file = h5py.File(test_birds[0], 'r')

In [12]:
wavfilenames = list(file['Box5_HP24/FD'].keys())

In [13]:
path2bird = test_birds[0]
dataset = bird_dataset(path2bird)

...total number of folders = 1 ...


In [14]:
dataset.folders

ItemsViewHDF5(<HDF5 file "Box5_HP24" (mode r)>)

In [ ]:
X_real = []

for n in range(dataset.nfolders):
    
    seqs = dataset.get(day = n, nsamps=-1)
    for s in seqs:
        segs = segment_image(s, width=16)
        segs = [np.ndarray.flatten(s) for s in segs]
        X_real += segs
        
        